In [49]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 1. Data load
- sample_songlist.csv : 가사 띄어쓰기가 잘 되어있는 데이터
- df_final_lyrics.csv : 노래마다 4가지 감정이 태깅되어 있는 데이터

In [6]:
song_origin=pd.read_csv("sample_songlist.csv",index_col=0)

In [7]:
song_origin.head()

,Title,Artist,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,Emotion,Proba
0,나의 X에게,경서,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,행복,44.40
1,취중고백,김민석 (멜로망스),발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,행복,61.14
2,사랑은 늘 도망가,임영웅,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,행복,77.44
3,듣고 싶을까,MSG워너비(M.O.M),발라드,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 함께 듣던 그 노랠 듣고 싶을까 듣고 나서 잠시 날 생각은 할까 아주 혹시라도...,행복,65.78
4,다정히 내 이름을 부르면,경서예지,발라드,끝없이 별빛이 내리던 밤기분 좋은 바람이두 빰을 스치고새벽 바다 한곳을 보는아름다운...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...,행복,57.01


In [8]:
song_origin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3552 entries, 0 to 3608
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  3552 non-null   object 
 1   Artist                 3552 non-null   object 
 2   Genre                  3552 non-null   object 
 3   Lyric                  3552 non-null   object 
 4   Lyric_ver2             3552 non-null   object 
 5   preprocess_Lyric_ver2  3552 non-null   object 
 6   Emotion                3552 non-null   object 
 7   Proba                  3552 non-null   float64
dtypes: float64(1), object(7)
memory usage: 249.8+ KB


In [9]:
song_tag=pd.read_csv("df_final_lyrics.csv",index_col=0)

In [10]:
song_tag.head()

,Title,SongID,Artist,Date,Genre,Lyrics,Emotion,Probability,분노혐오,놀람공포,슬픔,행복
0,ON,32399830.0,방탄소년단,2020.02.21,랩/힙합,"[""I can't understand what people are sayin'"", ...",행복,0.66,0.18,0.07,0.09,0.66
1,"어떤 날도, 어떤 말도",3534782.0,심규선 (Lucia),2011.09.07,"발라드, 인디음악","['함께 했던 많은 계절이', '봄에 눈이 녹듯 사라진다 해도', '아직 나는 너를...",슬픔,0.69,0.14,0.16,0.69,0.01
2,너 아니면 안돼,34592758.0,먼데이 키즈 (Monday Kiz),2022.02.03,발라드,"['오늘도 내 기억을 따라 헤매다', '이 길 끝에서 서성이는 나', '다신 볼 수...",슬픔,0.37,0.35,0.06,0.37,0.22
3,사랑이었나봐,33158202.0,기리보이,2020.12.23,랩/힙합,"['사소한 것들 땜에', '크게 다퉜었지', '하루 종일 말도 안 하고', '꿍해 ...",놀람공포,0.73,0.21,0.73,0.05,0.00
4,봄이야,34770461.0,로키드 (Lokid),2022.03.21,"R&B;/Soul, 인디음악","['서울은 좁아서', '어딜 가도 네가 따라와', '다시 돌아온 이 계절처럼', '...",놀람공포,0.60,0.16,0.60,0.18,0.06


In [11]:
#노래 & 가수가 같은 중복 데이터 제거
song_tag.drop_duplicates(subset=["Title","Artist"],inplace=True)
song_origin.drop_duplicates(subset=["Title","Artist"],inplace=True)

In [12]:
#인덱스 초기화
song_tag=song_tag.reset_index(drop=True)
song_origin=song_origin.reset_index(drop=True)

In [13]:
#노래 & 가수 열을 기준으로 데이터 합치기
song_total=pd.merge(song_tag,song_origin,on=["Title","Artist"],how="inner")

In [15]:
song_total.reset_index(drop=True,inplace=True)

In [16]:
song_total

,Title,SongID,Artist,Date,Genre_x,Lyrics,Emotion_x,Probability,분노혐오,놀람공포,슬픔,행복,Genre_y,Lyric,Lyric_ver2,preprocess_Lyric_ver2,Emotion_y,Proba
0,ON,32399830.0,방탄소년단,2020.02.21,랩/힙합,"[""I can't understand what people are sayin'"", ...",행복,0.66,0.18,0.07,0.09,0.66,랩/힙합,I can't understand what people are sayin'어느 장단...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",I can't understand what people are sayin' 어느 장...,분노혐오,38.29
1,"어떤 날도, 어떤 말도",3534782.0,심규선 (Lucia),2011.09.07,"발라드, 인디음악","['함께 했던 많은 계절이', '봄에 눈이 녹듯 사라진다 해도', '아직 나는 너를...",슬픔,0.69,0.14,0.16,0.69,0.01,"발라드, 인디음악",함께 했던 많은 계절이봄에 눈이 녹듯 사라진다 해도아직 나는 너를 기억해세상 무엇보...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",함께 했던 많은 계절이 봄에 눈이 녹듯 사라진다 해도 아직 나는 너를 기억해 세상 ...,행복,58.90
2,너 아니면 안돼,34592758.0,먼데이 키즈 (Monday Kiz),2022.02.03,발라드,"['오늘도 내 기억을 따라 헤매다', '이 길 끝에서 서성이는 나', '다신 볼 수...",슬픔,0.37,0.35,0.06,0.37,0.22,발라드,오늘도 내 기억을 따라 헤매다이 길 끝에서 서성이는 나다신 볼 수도 없는 네가 나를...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",오늘도 내 기억을 따라 헤매다 이 길 끝에서 서성이는 나 다신 볼 수도 없는 네가 ...,행복,57.76
3,사랑이었나봐,33158202.0,기리보이,2020.12.23,랩/힙합,"['사소한 것들 땜에', '크게 다퉜었지', '하루 종일 말도 안 하고', '꿍해 ...",놀람공포,0.73,0.21,0.73,0.05,0.00,랩/힙합,사소한 것들 땜에크게 다퉜었지하루 종일 말도 안 하고꿍해 있었지사랑이었나 봐그땐 몰...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",사소한 것들 땜에 크게 다퉜었지 하루 종일 말도 안 하고 꿍해 있었지 사랑이었나 봐...,행복,69.27
4,봄이야,34770461.0,로키드 (Lokid),2022.03.21,"R&B;/Soul, 인디음악","['서울은 좁아서', '어딜 가도 네가 따라와', '다시 돌아온 이 계절처럼', '...",놀람공포,0.60,0.16,0.60,0.18,0.06,"R&B;/Soul, 인디음악",서울은 좁아서어딜 가도 네가 따라와다시 돌아온 이 계절처럼또 너를 떠올려잘 해내고 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",서울은 좁아서 어딜 가도 네가 따라와 다시 돌아온 이 계절처럼 또 너를 떠올려 잘 ...,분노혐오,45.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353,"털어! (Feat. M1NU, Veinyfl, Swervy, 래원, 이영지, Lil...",33552228.0,BIG Naughty (서동현),2021.06.08,랩/힙합,"['은행 털어 Ay ', 'We the robbers ay', '지갑 헐어 버리고 ...",행복,0.87,0.02,0.05,0.05,0.87,랩/힙합,은행 털어 Ay We the robbers ay지갑 헐어 버리고 다시 벌어 Ay은행...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",은행 털어 Ay We the robbers ay 지갑 헐어 버리고 다시 벌어 Ay...,분노혐오,45.39
3354,대충 입고 나와,31835421.0,우디 (Woody),2019.05.29,랩/힙합,"['너는 너무너무 예뻐', '더할 나위가 없는걸', '내가 바보 같고 부족해도', ...",행복,0.78,0.11,0.03,0.08,0.78,랩/힙합,너는 너무너무 예뻐더할 나위가 없는걸내가 바보 같고 부족해도넌 내 옆에 있어네가 뭘...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",너는 너무너무 예뻐 더할 나위가 없는걸 내가 바보 같고 부족해도 넌 내 옆에 있어 ...,슬픔,43.21
3355,"천둥벌거숭이 (Feat. Jvcki Wai, 염따)",32071685.0,지코 (ZICO),2019.09.30,랩/힙합,"['그래서 어떻게 됐긴 어디도 안 가고', '내가 차린 회사랑 도장 찍었지', '빠...",놀람공포,0.57,0.03,0.57,0.20,0.19,랩/힙합,그래서 어떻게 됐긴 어디도 안 가고내가 차린 회사랑 도장 찍었지빠른 페이스로개체 수...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",그래서 어떻게 됐긴 어디도 안 가고 내가 차린 회사랑 도장 찍었지 빠른 페이스로 개...,분노혐오,58.35
3356,센치해,8038492.0,WINNER,2016.02.01,댄스,"['밤이 되면 어김없이 ', '찾아오는 이 느낌이 ', '왠지 원망스러워 ', '괜...",분노혐오,0.54,0.54,0.06,0.38,0.02,댄스,밤이 되면 어김없이 찾아오는 이 느낌이 왠지 원망스러워 괜히 또 전화 걸어 봐 너도...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",밤이 되면 어김없이 찾아오는 이 느낌이 왠지 원망스러워 괜히 또 전화 걸어 봐...,행복,52.96


In [18]:
song_total.columns

Index(['Title', 'SongID', 'Artist', 'Date', 'Genre_x', 'Lyrics', 'Emotion_x',
       'Probability', '분노혐오', '놀람공포', '슬픔', '행복', 'Genre_y', 'Lyric',
       'Lyric_ver2', 'preprocess_Lyric_ver2', 'Emotion_y', 'Proba'],
      dtype='object')

In [19]:
del song_total["Lyrics"]
del song_total["Date"]
del song_total["Genre_y"]
del song_total["Lyric_ver2"]
del song_total['Emotion_y']
del song_total['Proba']
del song_total["Lyric"]

In [20]:
song_total=song_total.rename(columns={"Genre_x":"Genre"})

In [22]:
song_total=song_total.rename(columns={"Emotion_x":"Emotion"})

In [23]:
song_total

,Title,SongID,Artist,Genre,Emotion,Probability,분노혐오,놀람공포,슬픔,행복,preprocess_Lyric_ver2
0,ON,32399830.0,방탄소년단,랩/힙합,행복,0.66,0.18,0.07,0.09,0.66,I can't understand what people are sayin' 어느 장...
1,"어떤 날도, 어떤 말도",3534782.0,심규선 (Lucia),"발라드, 인디음악",슬픔,0.69,0.14,0.16,0.69,0.01,함께 했던 많은 계절이 봄에 눈이 녹듯 사라진다 해도 아직 나는 너를 기억해 세상 ...
2,너 아니면 안돼,34592758.0,먼데이 키즈 (Monday Kiz),발라드,슬픔,0.37,0.35,0.06,0.37,0.22,오늘도 내 기억을 따라 헤매다 이 길 끝에서 서성이는 나 다신 볼 수도 없는 네가 ...
3,사랑이었나봐,33158202.0,기리보이,랩/힙합,놀람공포,0.73,0.21,0.73,0.05,0.00,사소한 것들 땜에 크게 다퉜었지 하루 종일 말도 안 하고 꿍해 있었지 사랑이었나 봐...
4,봄이야,34770461.0,로키드 (Lokid),"R&B;/Soul, 인디음악",놀람공포,0.60,0.16,0.60,0.18,0.06,서울은 좁아서 어딜 가도 네가 따라와 다시 돌아온 이 계절처럼 또 너를 떠올려 잘 ...
...,...,...,...,...,...,...,...,...,...,...,...
3353,"털어! (Feat. M1NU, Veinyfl, Swervy, 래원, 이영지, Lil...",33552228.0,BIG Naughty (서동현),랩/힙합,행복,0.87,0.02,0.05,0.05,0.87,은행 털어 Ay We the robbers ay 지갑 헐어 버리고 다시 벌어 Ay...
3354,대충 입고 나와,31835421.0,우디 (Woody),랩/힙합,행복,0.78,0.11,0.03,0.08,0.78,너는 너무너무 예뻐 더할 나위가 없는걸 내가 바보 같고 부족해도 넌 내 옆에 있어 ...
3355,"천둥벌거숭이 (Feat. Jvcki Wai, 염따)",32071685.0,지코 (ZICO),랩/힙합,놀람공포,0.57,0.03,0.57,0.20,0.19,그래서 어떻게 됐긴 어디도 안 가고 내가 차린 회사랑 도장 찍었지 빠른 페이스로 개...
3356,센치해,8038492.0,WINNER,댄스,분노혐오,0.54,0.54,0.06,0.38,0.02,밤이 되면 어김없이 찾아오는 이 느낌이 왠지 원망스러워 괜히 또 전화 걸어 봐...


In [25]:
song=song_total.copy()

In [26]:
# 장고에서 사용할 최종 데이터 (노래 선택 목록, 추천 목록)
#song.to_csv("song_total.csv")

## 2. 데이터 탐색

In [28]:
song["Genre"].value_counts()

댄스                        475
랩/힙합                      424
R&B/Soul                  401
성인가요/트로트                  385
발라드                       347
록/메탈                      305
인디음악, 포크/블루스              266
포크/블루스                    139
발라드, 인디음악                 119
인디음악, 록/메탈                114
R&B;/Soul, 인디음악            69
발라드, 국내드라마                 69
랩/힙합, 인디음악                 64
포크/블루스, 국내드라마              40
록/메탈, 국내드라마                40
R&B/Soul, 국내드라마            16
댄스, 랩/힙합                   12
인디음악, 일렉트로니카               11
발라드, R&B/Soul               9
성인가요/트로트, 국내드라마             9
발라드, 록/메탈                   6
발라드, R&B/Soul, 국내드라마        5
랩/힙합, 국내드라마                 5
발라드, 랩/힙합, 국내드라마            4
댄스, 인디음악                    3
록/메탈, 애니메이션/웹툰              3
발라드, 포크/블루스                 2
댄스, 국내드라마                   2
발라드, 게임                     2
록/메탈, 국내영화                  2
발라드, 성인가요/트로트               1
록/메탈, 애니메이션/웹툰, 키즈, 만화      1
발라드, 인디음악, 포크/블루스           1
발라드, 국내영화 

In [29]:
song["Emotion"].value_counts()

행복      1463
슬픔       850
놀람공포     658
분노혐오     387
Name: Emotion, dtype: int64

In [47]:
song[song["Emotion"]=="행복"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포   슬픔   행복  \
Emotion Genre                                                                
행복      댄스          271     256     271          271   271   271  271  271   
        랩/힙합        206     199     206          206   206   206  206  206   
        R&B/Soul    200     187     200          200   200   200  200  200   
        성인가요/트로트    138     134     138          138   138   138  138  138   
        록/메탈        134     122     134          134   134   134  134  134   
        발라드         111     108     111          111   111   111  111  111   

                  preprocess_Lyric_ver2  
Emotion Genre                            
행복      댄스                          271  
        랩/힙합                        206  
        R&B/Soul                    200  
        성인가요/트로트                    138  
        록/메탈                        134  
        발라드                         111

In [45]:
song[song["Emotion"]=="놀람공포"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포  슬픔  행복  \
Emotion Genre                                                              
놀람공포    성인가요/트로트     96      96      96           96    96    96  96  96   
        댄스           92      87      92           92    92    92  92  92   
        랩/힙합         83      80      83           83    83    83  83  83   
        R&B/Soul     70      64      70           70    70    70  70  70   
        발라드          62      60      62           62    62    62  62  62   
        록/메탈         61      60      61           61    61    61  61  61   

                  preprocess_Lyric_ver2  
Emotion Genre                            
놀람공포    성인가요/트로트                     96  
        댄스                           92  
        랩/힙합                         83  
        R&B/Soul                     70  
        발라드                          62  
        록/메탈                         61

In [46]:
song[song["Emotion"]=="슬픔"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포   슬픔  \
Emotion Genre                                                               
슬픔      발라드             132     128     132          132   132   132  132   
        성인가요/트로트        107     106     107          107   107   107  107   
        R&B/Soul         96      96      96           96    96    96   96   
        록/메탈             82      79      82           82    82    82   82   
        인디음악, 포크/블루스     81      73      81           81    81    81   81   
        랩/힙합             72      69      72           72    72    72   72   

                       행복  preprocess_Lyric_ver2  
Emotion Genre                                     
슬픔      발라드           132                    132  
        성인가요/트로트      107                    107  
        R&B/Soul       96                     96  
        록/메탈           82                     82  
        인디음악, 포크/블루스   81                     81  
        랩/힙합           72                     72

In [48]:
song[song["Emotion"]=="분노혐오"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포  슬픔  행복  \
Emotion Genre                                                                  
분노혐오    랩/힙합             63      58      63           63    63    63  63  63   
        성인가요/트로트         44      43      44           44    44    44  44  44   
        댄스               43      42      43           43    43    43  43  43   
        발라드              42      41      42           42    42    42  42  42   
        인디음악, 포크/블루스     37      34      37           37    37    37  37  37   
        R&B/Soul         35      33      35           35    35    35  35  35   

                      preprocess_Lyric_ver2  
Emotion Genre                                
분노혐오    랩/힙합                             63  
        성인가요/트로트                         44  
        댄스                               43  
        발라드                              42  
        인디음악, 포크/블루스                     37  
        R&B/Soul                         35

## 3. 곡 유사도 구하기
- 장르 유사도 by Countervectorizer
- 가사 유사도 by Tfidfvectorizer
+ 가수 유사도를 포함 시키면 같은 가수로 편향되어 뺌

### 3-1. Countervectorizer를 이용하여 장르 유사도 구하기

In [50]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_category = count_vect_category.fit_transform(song["Genre"]) 
song_simi_genre = cosine_similarity(genre_category, genre_category) 

In [51]:
cosine_similarity(genre_category, genre_category).mean()

0.13355077264348325

In [58]:
genre_result=pd.DataFrame(song_simi_genre)
genre_result.columns = song["Genre"]
genre_result.index = song["Genre"]
genre_result.head()

Genre,랩/힙합,"발라드, 인디음악",발라드,랩/힙합,"R&B;/Soul, 인디음악",R&B/Soul,댄스,댄스,성인가요/트로트,R&B/Soul,...,"인디음악, 포크/블루스","인디음악, 포크/블루스",랩/힙합,"인디음악, 록/메탈","인디음악, 포크/블루스",랩/힙합,랩/힙합,랩/힙합,댄스,성인가요/트로트
Genre,,,,,,,,,,,,,,,,,,,,,
랩/힙합,1.0,0.000000,0.00000,1.0,0.000000,0.00000,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,1.0,0.000000,0.000000,1.0,1.0,1.0,0.0,0.0
"발라드, 인디음악",0.0,1.000000,0.57735,0.0,0.333333,0.00000,0.0,0.0,0.0,0.00000,...,0.258199,0.258199,0.0,0.333333,0.258199,0.0,0.0,0.0,0.0,0.0
발라드,0.0,0.577350,1.00000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
랩/힙합,1.0,0.000000,0.00000,1.0,0.000000,0.00000,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,1.0,0.000000,0.000000,1.0,1.0,1.0,0.0,0.0
"R&B;/Soul, 인디음악",0.0,0.333333,0.00000,0.0,1.000000,0.57735,0.0,0.0,0.0,0.57735,...,0.258199,0.258199,0.0,0.333333,0.258199,0.0,0.0,0.0,0.0,0.0


### 3-2. TfidfVectorizer 이용하여 가사 유사도 구하기

In [54]:
vectorizer = TfidfVectorizer()
tf_lyric = vectorizer.fit_transform(song["preprocess_Lyric_ver2"]).todense()
song_simi_lyric = cosine_similarity(tf_lyric, tf_lyric) 

lyric_result=pd.DataFrame(song_simi_lyric)
lyric_result.columns = song["preprocess_Lyric_ver2"]
lyric_result.index = song["preprocess_Lyric_ver2"]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [57]:
lyric_result.head()

preprocess_Lyric_ver2                               I can't understand what people are sayin' 어느 장단에 맞춰야 될지 한 발자국 떼면 한 발자국 커지는 shadow 잠에서 눈을 뜬 여긴 또 어디 어쩜 서울 또 New York or Paris 일어나니 휘청이는 몸  Look at my feet, look down 날 닮은 그림자 흔들리는 건 이놈인가 아니면 내 작은 발끝인가 두렵잖을 리 없잖아 다 괜찮을 리 없잖아 그래도 I know 서툴게 I flow 저 까만 바람과 함께 날아  Hey na na na 미치지 않으려면 미쳐야 해 Hey na na na 나를 다 던져 이 두 쪽 세상에 Hey na na na Can’t hold me down cuz you know I’m a fighter 제 발로 들어온 아름다운 감옥 Find me and I'm gonna live with ya  (Eh-oh) 가져와 bring the pain oh yeah (Eh-oh) 올라타봐 bring the pain oh yeah Rain be pourin' Sky keep fallin' Everyday oh na na na (Eh-oh) 가져와 bring the pain oh yeah  Bring the pain 모두 내 피와 살이 되겠지 Bring the pain No fear, 방법을 알겠으니 작은 것에 breathe 그건 어둠 속 내 산소와 빛 내가 나이게 하는 것들의 힘 넘어져도 다시 일어나 scream  넘어져도 다시 일어나 scream 언제나 우린 그랬으니 설령 내 무릎이 땅에 닿을지언정 파묻히지 않는 이상 그저 그런 해프닝쯤 될 거란 걸 Win no matter what Win no matter what Win no matter what 네가 뭐라던 누가 뭐라던 I don't give a uhh I don't give a uhh I don't give a uhh  Hey na na na 미치지 않으려면 미쳐야 해 Hey na na na 나를 다 던져 이 두 쪽 세상에 Hey na na na Can’t hold me down cuz you know I’m a fighter 제 발로 들어온 아름다운 감옥 Find me and I'm gonna live with ya  (Eh-oh) 가져와 bring the pain oh yeah (Eh-oh) 올라타봐 bring the pain oh yeah Rain be pourin' Sky keep fallin' Everyday oh na na na (Eh-oh) 가져와 bring the pain oh yeah  나의 고통이 있는 곳에 내가 숨 쉬게 하소서 My everythin’ My blood and tears Got no fears I'm singin’ ohhhhh Oh I’m takin’ over You should know yeah Can’t hold me down cuz you know I’m a fighter 깜깜한 심연 속 기꺼이 잠겨 Find me and I’m gonna bleed with ya  (Eh-oh) 가져와 bring the pain oh yeah (Eh-oh) 올라타봐 bring the pain oh yeah Rain be pourin' Sky keep fallin' Everyday oh na na na (Eh-oh) Find me and I’m gonna bleed with ya  (Eh-oh) 가져와 bring the pain oh yeah (Eh-oh) 올라타봐 bring the pain oh yeah All that I know is just goin’ on &amp; on &amp; on &amp; on (Eh-oh) 가져와 bring the pain oh yeah\r\n\t\t\t\t\t</div>  \
preprocess_Lyric_ver2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
I can't understand what people are sayin' 어느 장단...                                           1.000000                                                                                                 

In [55]:
song_simi_lyric.mean()

0.012733605365829086

### 3-3 곡 유사도 구하는 공식 생성

In [59]:
song_simi_co = (
                 + song_simi_genre * 0.3 # 가중치 1. 장르 유사도
                 + song_simi_lyric * 1 # 가중치 2. 가사 유사도
                 )

In [60]:
song_simi_co_sorted_ind = song_simi_co.argsort()[:, ::-1] 
song_simi_co_sorted_df=pd.DataFrame(song_simi_co_sorted_ind)

In [61]:
# 행을 기준으로 가장 유사도 값이 큰 순으로 인덱스 저장
song_simi_co_sorted_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357
0,0,2643,858,38,2862,1358,3088,2982,442,810,...,2919,3277,1978,1977,3323,1950,1938,2619,1928,1897
1,1,2524,304,429,479,1494,2468,3118,1805,1181,...,2154,2756,326,2150,2755,1791,333,1801,1812,2260
2,2,1712,1903,607,1633,1341,1326,848,1588,906,...,1970,357,1700,2804,1696,842,1454,709,1033,642
3,3,344,1742,1581,3344,2680,111,1169,2224,2048,...,2644,1014,1029,1017,2636,1020,2633,2632,1025,3357
4,4,1645,15,2176,2156,338,1398,3245,1585,1873,...,255,1579,2658,2366,2364,1305,873,1191,1949,2545


In [62]:
# 사용자가 선택한 노래와 유사한 노래를 찾기 위한 데이터 (장고에서 사용)
# song_simi_co_sorted_df.to_csv("song_similarity.csv")

## + 장고에서 사용할 코드

In [63]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def find_simi_song(seed_song,top_n=10):

    song_df = pd.read_csv("song_total.csv",index_col=0)
    song_simi_co_sorted_df = pd.read_csv("song_similarity.csv",index_col=0)
    song_simi_co_sorted = np.array(song_simi_co_sorted_df)

    song_recom=pd.DataFrame(columns=song_df.columns)

    for seed in seed_song:
        title=song_df[song_df["Title"] == seed]
        title_index=title.index.values
        similar_indexes = song_simi_co_sorted[title_index,:top_n]
        similar_indexes = similar_indexes.reshape(-1)
        song_recom=pd.concat([song_recom,song_df.iloc[similar_indexes].iloc[1:,:]])

    return song_recom

def user_song_simi(song_recom,total_array):

    song_emotion=np.array(song_recom[['분노혐오', '놀람공포', '슬픔', '행복']])

    # total_array 문자열 -> numpy 배열
    total_array_list=list(total_array[2:-2].split())
    total_array_list=list(map(float,total_array_list))
    total_array=np.array([total_array_list])

    song_user_cosine_index=cosine_similarity(total_array,song_emotion).argsort()[:, ::-1]
    song_recom_index=song_user_cosine_index.reshape(-1)[:5]


    return song_recom.iloc[song_recom_index]

def random_recom():
    song_df = pd.read_csv("song_total.csv",index_col=0)
    random_df = song_df.sample(n=5, replace=False)

    return random_df

In [64]:
#사용자가 선택한 노래 목록
seed_song=["취중고백","Madeleine Love","어머님 사랑합니다"]

In [65]:
#사용자가 선택한 노래 제외하고 5곡씩 유사한 곡 추출 (3곡 선택 * 유사한 5곡 -> 15곡 추출)
song_recom=find_simi_song(seed_song,6)
song_recom

,Title,SongID,Artist,Genre,Emotion,Probability,분노혐오,놀람공포,슬픔,행복,preprocess_Lyric_ver2
2393,사랑했었다 (Still love you),31004503.0,이홍기 (FT아일랜드),발라드,놀람공포,0.52,0.04,0.52,0.12,0.32,잊지 마 잊지 마 우리 헤어질 때 힘들던 그날의 인사를 울지 마 울지 마 부디 행복...
1668,못해준 게 많아서,34598809.0,잠골버스,발라드,놀람공포,0.91,0.02,0.91,0.01,0.07,몇 달이 지나가도 참 보고 싶은 너 아름다웠지 우리의 추억들 지나간 시간만큼 더 보...
1147,멜로를 좋아해,34978897.0,포맨 (4MEN),발라드,행복,0.77,0.03,0.01,0.19,0.77,왜 그땐 몰랐을까 영화를 볼 때마다 왜 그땐 몰랐을까 늘 멜로만 고집한 너 말하지 ...
423,미친 것처럼,35126568.0,V.O.S,발라드,분노혐오,0.64,0.64,0.11,0.25,0.01,너는 참 못됐다 나 없이도 잘 지내더라 언제쯤 나도 널 잊을 수 있을까 눈을 감아도...
1118,태양계,NaN,성시경,발라드,슬픔,0.89,0.02,0.02,0.89,0.08,나의 사랑이 멀어지네 나의 어제는 사라지네 태양을 따라 도는 저 별들처럼 난 돌고 ...
1375,사랑하는 너에게 (취향저격 그녀 X 스탠딩 에그),32993976.0,스탠딩 에그,"발라드, 인디음악",행복,0.67,0.07,0.07,0.19,0.67,날 보는 두 눈은 사랑으로 가득 차 있어 그대 손을 잡고 오늘도 난 웃어요 널 보는...
3282,여자친구,31652482.0,하은요셉,"발라드, 인디음악",행복,0.93,0.02,0.02,0.03,0.93,나의 여자친구가 되어주겠니 이런 내 마음을 넌 받아주겠니 넌 누구보다 예뻐 넌 뭘 ...
2137,예뻐서 (Feat. 루이 Of 긱스) (You're beautiful),5795006.0,유승우,"발라드, 인디음악",놀람공포,0.38,0.15,0.38,0.13,0.34,예뻐서 그 얼굴이 너무 예뻐서 준비해놨던 얘기들 또 못했어 니가 웃을 땐 온 세상이...
1744,바라봐줘요,31624517.0,죠지,"발라드, 인디음악",슬픔,0.54,0.09,0.10,0.54,0.27,그대를 다시 볼 수 있다는 생각을 해요 아마도 우린 끝없이 멀리 돌아만 왔죠 엊그제...
2291,우우우린,NaN,윤지영,"발라드, 인디음악",슬픔,0.94,0.03,0.01,0.94,0.02,너와 나 지금이 아니어도 괜찮았을 거야 우린 외롭지 않았을 거야 멀리서 우리 손을 ...


In [66]:
#사용자가 입력한 text에 대한 4가지 감정 확률값
total_array="[[0.61 0.20 0.13 0.06]]"

#최종 사용자에게 추천해줄 5곡
song_recomm_final=user_song_simi(song_recom,total_array)
song_recomm_final

,Title,SongID,Artist,Genre,Emotion,Probability,분노혐오,놀람공포,슬픔,행복,preprocess_Lyric_ver2
423,미친 것처럼,35126568.0,V.O.S,발라드,분노혐오,0.64,0.64,0.11,0.25,0.01,너는 참 못됐다 나 없이도 잘 지내더라 언제쯤 나도 널 잊을 수 있을까 눈을 감아도...
2976,회룡포,33207714.0,김다현,성인가요/트로트,분노혐오,0.35,0.35,0.20,0.32,0.13,내 것이 아닌 것을 멀리 찾아서 휘돌아 감은 거 그 세월이 얼마이더냐 물설고 낮설은...
1675,사모곡,33279089.0,양지은,성인가요/트로트,슬픔,0.36,0.30,0.24,0.36,0.11,앞산 노을 질때까지 호미자루 벗을 삼아 화전밭 일구시고 흙에 살던 어머니 땀에 찌든...
2137,예뻐서 (Feat. 루이 Of 긱스) (You're beautiful),5795006.0,유승우,"발라드, 인디음악",놀람공포,0.38,0.15,0.38,0.13,0.34,예뻐서 그 얼굴이 너무 예뻐서 준비해놨던 얘기들 또 못했어 니가 웃을 땐 온 세상이...
2393,사랑했었다 (Still love you),31004503.0,이홍기 (FT아일랜드),발라드,놀람공포,0.52,0.04,0.52,0.12,0.32,잊지 마 잊지 마 우리 헤어질 때 힘들던 그날의 인사를 울지 마 울지 마 부디 행복...


In [67]:
#사용자가 아무것도 선택하지 않을 시 랜덤으로 5곡 추천
random_recom()

,Title,SongID,Artist,Genre,Emotion,Probability,분노혐오,놀람공포,슬픔,행복,preprocess_Lyric_ver2
1981,듣고 자요,30138513.0,그_냥,"인디음악, 포크/블루스",행복,0.84,0.02,0.05,0.09,0.84,오늘 그대의 하루는 길었죠 이런 저런 일도 많이 있었죠 투정부리며 내게 털어놔요 예...
1632,Run Devil Run,NaN,소녀시대 (GIRLS' GENERATION),댄스,놀람공포,0.91,0.04,0.91,0.05,0.00,똑바로 해 넌 정말 Bad boy\r 사랑보단 호기심뿐 그 동안 난\r 너 땜에 깜...
2409,Killing Me,34335697.0,청하,댄스,분노혐오,0.38,0.38,0.04,0.36,0.21,Everything feels so strange 바뀐 일상 속에 문을 닫고 맘을 ...
1354,너에게 난 나에게 넌,32719279.0,임영웅,성인가요/트로트,행복,0.86,0.03,0.01,0.10,0.86,너에게 난 해질녘 노을처럼 한편의 아름다운 추억이 되고 소중했던 우리 푸르던 날을 ...
2122,좋아해.. (바른연애 길잡이 X 죠지),33293837.0,죠지,"발라드, 인디음악",놀람공포,0.87,0.06,0.87,0.06,0.01,용기 내어보려 노력했는데 왜 네 앞에서는 다 어색해지고 꽤 자연스럽던 내 인사마저도...
